In [1]:
%%html

<style>
th {
    text-align: left;
}
</style>

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

from depsurf import FileLogger, check_diff, OUTPUT_PATH, Kind, LinuxImage


def get_lsm(btf):
    return {
        f"security_{e['name']}"
        for e in btf.get(Kind.STRUCT, "security_hook_heads")["members"]
    }


def diff_btf(img1: LinuxImage, img2: LinuxImage, group, name):
    path = OUTPUT_PATH / group / name
    print(f"Results saving to {path}")

    results = {}
    for key, kind in [
        ("func", Kind.FUNC),
        ("struct", Kind.STRUCT),
        # ("union", Kind.UNION),
        # ("enum", Kind.ENUM),
        ("tp_event", Kind.STRUCT),
        ("tp_func", Kind.FUNC),
        ("lsm", Kind.FUNC),
    ]:
        if key == "lsm":
            dict1 = img1.lsm_hooks
            dict2 = img2.lsm_hooks
        elif key == "tp_func":
            dict1 = img1.tracepoints.funcs
            dict2 = img2.tracepoints.funcs
        elif key == "tp_event":
            dict1 = img1.tracepoints.events
            dict2 = img2.tracepoints.events
        else:
            dict1 = img1.btf.get_all_kind(kind)
            dict2 = img2.btf.get_all_kind(kind)

        output_path = path / f"{key}.log"
        with FileLogger(output_path):
            diff_result = check_diff(dict1, dict2, kind)
        results[(key, "Old")] = len(dict1)
        results[(key, "New")] = len(dict2)
        results[(key, "Common")] = len(diff_result.common)
        results[(key, "Added")] = len(diff_result.added)
        results[(key, "Removed")] = len(diff_result.removed)
        results[(key, "Changed")] = len(diff_result.changed)
        for k, v in diff_result.reasons.items():
            results[(key, f"- {k}")] = v

    df = pd.DataFrame(
        results.values(),
        index=pd.MultiIndex.from_tuples(results.keys()),
        columns=[name],
    )
    df.to_string(path / "summary.txt")
    df.to_pickle(path / "summary.pkl")
    return df


diff_btf(
    LinuxImage.from_str("5.4.0-26-generic-amd64"),
    LinuxImage.from_str("5.8.0-25-generic-amd64"),
    "dist",
    "test",
)

[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-amd64.pkl


Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/test


[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.8.0-25-generic-amd64.pkl
[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-amd64.pkl
[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.8.0-25-generic-amd64.pkl


test
func     Old                    48018
         New                    51936
         Common                 45151
         Added                   6785
         Removed                 2867
         Changed                  871
         - Param added            540
         - Param removed          433
         - Param type changed     173
         - Param reordered        138
         - Return type changed    120
struct   Old                     8422
         New                     8610
         Common                  8130
         Added                    480
         Removed                  292
         Changed                  896
         - Field added            477
         - Field removed          238
         - Field type changed     194
         - Layout changed           0
tp_event Old                      752
         New                      785
         Common                   740
         Added                     45
         Removed                   12
         Changed                   51
         - Field added             12
         - Field removed           11
         - Field type changed      30
         - Layout changed           0
tp_func  Old                      752
         New                      785
         Common                   740
         Added                     45
         Removed                   12
         Changed                   25
         - Param added             14
         - Param removed           15
         - Param type changed       2
         - Param reordered          7
         - Return type changed      0
lsm      Old                      189
         New                      198
         Common                   188
         Added                     10
         Removed                    1
         Changed                   14
         - Param added             13
         - Param removed           13
         - Param type changed       2
         - Param reordered          0
         - Return type changed      0

In [4]:
from depsurf import BuildVersion

imgs = [bv.get_img() for bv in BuildVersion.iter(flavor="generic", arch="amd64")]

In [6]:
from itertools import pairwise

results = {}
for img1, img2 in pairwise(imgs):
    print(f"Comparing {img1} and {img2}")
    name = f"{img1.version}|{img2.version}"
    results[name] = diff_btf(img1, img2, "dist", name)

Comparing LinuxImage(4.4.0-21-generic-amd64) and LinuxImage(4.8.0-22-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.4.0-21-generic-amd64-4.8.0-22-generic-amd64
Comparing LinuxImage(4.8.0-22-generic-amd64) and LinuxImage(4.10.0-19-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.8.0-22-generic-amd64-4.10.0-19-generic-amd64
Comparing LinuxImage(4.10.0-19-generic-amd64) and LinuxImage(4.13.0-16-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.10.0-19-generic-amd64-4.13.0-16-generic-amd64
Comparing LinuxImage(4.13.0-16-generic-amd64) and LinuxImage(4.15.0-20-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.13.0-16-generic-amd64-4.15.0-20-generic-amd64
Comparing LinuxImage(4.15.0-20-generic-amd64) and LinuxImage(4.18.0-10-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.15.0-20-generic-amd64-4.18.0-10-generic-am

[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.18.0-10-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.0.0-13-generic-amd64.pkl


Comparing LinuxImage(4.18.0-10-generic-amd64) and LinuxImage(5.0.0-13-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.18.0-10-generic-amd64-5.0.0-13-generic-amd64


[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.0.0-13-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.3.0-18-generic-amd64.pkl


Comparing LinuxImage(5.0.0-13-generic-amd64) and LinuxImage(5.3.0-18-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/5.0.0-13-generic-amd64-5.3.0-18-generic-amd64


...

[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.19.0-21-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/6.2.0-20-generic-amd64.pkl


Comparing LinuxImage(5.19.0-21-generic-amd64) and LinuxImage(6.2.0-20-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/5.19.0-21-generic-amd64-6.2.0-20-generic-amd64


[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/6.2.0-20-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/6.5.0-9-generic-amd64.pkl


Comparing LinuxImage(6.2.0-20-generic-amd64) and LinuxImage(6.5.0-9-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/6.2.0-20-generic-amd64-6.5.0-9-generic-amd64


[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/6.5.0-9-generic-amd64.pkl


In [9]:
df = pd.concat(results, axis=1)
df.columns = df.columns.droplevel(2)
df = df.loc[(df != 0).any(axis=1)]
df.to_pickle("output/diff.pkl")

df

4.4.0-21-generic-amd64  4.8.0-22-generic-amd64  \
                               4.8.0-22-generic-amd64 4.10.0-19-generic-amd64   
func     Old                                    36309                   38211   
         New                                    38211                   39148   
         Common                                 35362                   37574   
         Added                                   2849                    1574   
         Removed                                  947                     637   
         Changed                                  768                     386   
         - Param added                            404                     221   
         - Param removed                          292                     178   
         - Param type changed                     212                     107   
         - Param reordered                        173                      42   
         - Return type changed                    114                      54   
struct   Old                                     6160                    6599   
         New                                     6599                    6752   
         Common                                  6063                    6525   
         Added                                    536                     227   
         Removed                                   97                      74   
         Changed                                  759                     646   
         - Field added                            408                     249   
         - Field removed                          222                     101   
         - Field type changed                     144                     117   
tp_event Old                                      502                     539   
         New                                      539                     559   
         Common                                   502                     531   
         Added                                     37                      28   
         Removed                                    0                       8   
         Changed                                   29                      17   
         - Field added                             25                       2   
         - Field removed                           19                      14   
         - Field type changed                       0                       1   
tp_func  Old                                      502                     539   
         New                                      539                     559   
         Common                                   502                     531   
         Added                                     37                      28   
         Removed                                    0                       8   
         Changed                                   14                       2   
         - Param added                             10                       1   
         - Param removed                            4                       0   
         - Param type changed                       0                       1   
         - Param reordered                          4                       0   
lsm      Old                                      168                     168   
         New                                      168                     171   
         Common                                   165                     168   
         Added                                      3                       3   
         Removed                                    3                       0   
         Changed                                   16                       0   
         - Param added                              0                       0   
         - Param removed                            0                       0   
         - Param type changed                      16         

In [ ]:
import pandas as pd

pd.set_option("display.max_rows", None)

df = pd.read_pickle("output/diff.pkl")
df = df.astype(object)

kinds = ["struct", "func", "trace_event", "lsm"]
groups = ["20.04"]
df = df.loc[kinds, groups]

for kind in kinds:
    for row in df.loc[kind, :].index:
        if row == "Old":
            continue
        for col in df.columns:
            val = df.loc[(kind, row), col]
            percentage = val / df.loc[(kind, "Old"), col] * 100
            df.loc[(kind, row), col] = f"\\val{{{val}}}{{{percentage:4.1f}}}"

df = df.drop(index=["Old", "New", "Common"], level=1)

df.to_latex("output/diff.tex")

df

20.04                    \
                                           5.4->5.8         5.8->5.11   
struct      Added                   \val{529}{ 6.3}   \val{513}{ 5.9}   
            Removed                 \val{203}{ 2.4}   \val{156}{ 1.8}   
            Changed                 \val{973}{11.6}   \val{771}{ 8.8}   
            - Field added           \val{485}{ 5.8}   \val{439}{ 5.0}   
            - Field removed         \val{315}{ 3.8}   \val{232}{ 2.7}   
            - Field type changed    \val{219}{ 2.6}   \val{164}{ 1.9}   
            - Layout changed        \val{897}{10.7}   \val{720}{ 8.3}   
func        Added                  \val{3564}{ 7.4}  \val{3080}{ 6.2}   
            Removed                \val{2000}{ 4.2}  \val{1076}{ 2.2}   
            Changed                 \val{932}{ 1.9}  \val{1176}{ 2.4}   
            - Param added           \val{535}{ 1.1}   \val{375}{ 0.8}   
            - Param removed         \val{475}{ 1.0}   \val{343}{ 0.7}   
            - Param type changed    \val{208}{ 0.4}   \val{623}{ 1.3}   
            - Param reordered       \val{132}{ 0.3}   \val{163}{ 0.3}   
            - Return type changed   \val{130}{ 0.3}    \val{82}{ 0.2}   
trace_event Added                    \val{51}{11.0}    \val{31}{ 6.2}   
            Removed                  \val{16}{ 3.4}     \val{6}{ 1.2}   
            Changed                  \val{36}{ 7.7}    \val{14}{ 2.8}   
            - Field added             \val{8}{ 1.7}    \val{11}{ 2.2}   
            - Field removed           \val{8}{ 1.7}     \val{3}{ 0.6}   
            - Field type changed     \val{25}{ 5.4}     \val{0}{ 0.0}   
            - Layout changed         \val{31}{ 6.7}    \val{14}{ 2.8}   
lsm         Added                    \val{10}{ 5.3}     \val{1}{ 0.5}   
            Removed                   \val{1}{ 0.5}     \val{0}{ 0.0}   
            Changed                  \val{18}{ 9.5}     \val{5}{ 2.5}   
            - Param added            \val{13}{ 6.9}     \val{4}{ 2.0}   
            - Param removed          \val{13}{ 6.9}     \val{2}{ 1.0}   
            - Param type changed      \val{6}{ 3.2}     \val{1}{ 0.5}   
            - Param reordered         \val{0}{ 0.0}     \val{0}{ 0.0}   

                                                                       \
                                         5.11->5.13        5.13->5.15   
struct      Added                   \val{233}{ 2.6}   \val{283}{ 3.1}   
            Removed                  \val{96}{ 1.1}   \val{117}{ 1.3}   
            Changed                 \val{476}{ 5.2}   \val{656}{ 7.1}   
            - Field added           \val{272}{ 3.0}   \val{365}{ 4.0}   
            - Field removed         \val{154}{ 1.7}   \val{163}{ 1.8}   
            - Field type changed    \val{111}{ 1.2}   \val{140}{ 1.5}   
            - Layout changed        \val{438}{ 4.8}   \val{605}{ 6.6}   
func        Added                  \val{1343}{ 2.6}  \val{2121}{ 4.1}   
            Removed                 \val{755}{ 1.5}  \val{1123}{ 2.1}   
            Changed                 \val{549}{ 1.1}   \val{640}{ 1.2}   
            - Param added           \val{428}{ 0.8}   \val{298}{ 0.6}   
            - Param removed         \val{172}{ 0.3}   \val{194}{ 0.4}   
            - Param type changed     \val{42}{ 0.1}   \val{214}{ 0.4}   
            - Param reordered       \val{280}{ 0.5}   \val{104}{ 0.2}   
            - Return type changed    \val{51}{ 0.1}   \val{111}{ 0.2}   
trace_event Added                    \val{13}{ 2.5}    \val{17}{ 3.3}   
            Removed                  \val{15}{ 2.9}    \val{15}{ 2.9}   
            Changed                  \val{10}{ 1.9}    \val{17}{ 3.3}   
            - Field added             \val{6}{ 1.1}     \val{6}{ 1.1}   
            - Field removed           \val{1}{ 0.2}     \val{2}{ 0.4}   
            - Field type changed      \val{5}{ 1.0}    \val{10}{ 1.9}   
            - Layout changed          \val{9}{ 1.7}    \val{14}{ 2.7}   
lsm         Added             